In [1]:
# =====================================================================
# VALOR-P: (A) Poisson simples  |  (B) Regressão Poisson (GLM)
# =====================================================================

set.seed(123)

# =====================================================================
# A) POISSON SIMPLES — valor-p condicional a H0
#    H0: theta = theta0
#    Estatística: Tn = n * ( (mean(z) - theta0)^2 / var(z) )  ~ qui^2_1 (aprox.)
# =====================================================================

# ---- função: gera p-valores sob diferentes condições -----------------
sim_p_poisson = function(n, theta0, M, violacao = "nenhuma") {
  # violacao = "nenhuma" | "dependencia" | "hetero"
  p = numeric(M)

  for (m in 1:M) {
    if (violacao == "nenhuma") {
      z = rpois(n, lambda = theta0)
    }
    if (violacao == "dependencia") {
      # ruído comum -> correlação positiva entre observações
      u = rnorm(1, 0, 0.25)
      z = rpois(n, lambda = pmax(1e-8, theta0 * exp(u)))  # média marginal ~ theta0
    }
    if (violacao == "hetero") {
      # heterocedasticidade: metades com variâncias distintas (meios com taxas diferentes)
      z = c(
        rpois(n/2, lambda = theta0 * 0.7),
        rpois(n - n/2, lambda = theta0 * 1.3)
      )
    }

    Tn = n * ( (mean(z) - theta0)^2 / var(z) )
    p[m] = 1 - pchisq(Tn, df = 1)  # p assintótico
  }

  return(p)
}

# ---- parâmetros e execução -------------------------------------------
n      = 200
theta0 = 2
M      = 10000
alpha  = 0.05

p_h0        = sim_p_poisson(n, theta0, M, violacao = "nenhuma")
p_dep       = sim_p_poisson(n, theta0, M, violacao = "dependencia")
p_hetero    = sim_p_poisson(n, theta0, M, violacao = "hetero")

# ---- checagens simples (uniformidade sob H0) -------------------------
# sob H0 ideal: E[1{p <= a}] ~ a  -> aqui checamos alguns pontos
checagem = data.frame(
  a        = c(0.01, 0.05, 0.10, 0.50),
  H0_ideal = c(0.01, 0.05, 0.10, 0.50),
  H0_emp   = sapply(c(0.01, 0.05, 0.10, 0.50), function(a) mean(p_h0 <= a)),
  dep_emp  = sapply(c(0.01, 0.05, 0.10, 0.50), function(a) mean(p_dep <= a)),
  hetero_emp = sapply(c(0.01, 0.05, 0.10, 0.50), function(a) mean(p_hetero <= a))
)
print(round(checagem, 3))

# =====================================================================
# B) REGRESSÃO POISSON (GLM)
#     log(lambda_i) = beta0 + beta1 * x_i
#     Teste H0: beta1 = 0  (Wald e Razão de Verossimilhanças)
# =====================================================================

# ---- função auxiliar: ajusta GLM e computa p (Wald e LR) -------------
teste_glm_poisson = function(y, x) {
  modelo_completo = glm(y ~ x, family = poisson)
  modelo_reduzido = glm(y ~ 1, family = poisson)

  # Wald
  b1   = coef(modelo_completo)[2]
  se1  = sqrt(vcov(modelo_completo)[2, 2])
  W    = (b1 / se1)^2
  p_W  = 1 - pchisq(W, df = 1)

  # Razão de verossimilhanças
  LR   = 2 * (as.numeric(logLik(modelo_completo)) - as.numeric(logLik(modelo_reduzido)))
  p_LR = 1 - pchisq(LR, df = 1)

  data.frame(
    p_Wald = p_W,
    p_LR   = p_LR,
    beta1_estimado = b1,
    erro_beta1     = se1,
    estatistica_W  = W,
    estatistica_LR = LR
  )
}

# ---- EXEMPLOS NUMÉRICOS — três cenários (um único conjunto por cenário)
set.seed(123)
n     = 200
beta0 = 1
alpha = 0.05
x     = runif(n, 0, 2)   # mesma covariável p/ todos os cenários

exemplo_cenario = function(beta1_real, rotulo) {
  lambda = exp(beta0 + beta1_real * x)
  y      = rpois(n, lambda)
  out    = teste_glm_poisson(y, x)

  # decide por Wald (poderia escolher LR; aqui mantemos Wald)
  decisao = ifelse(out$p_Wald < alpha, "Rejeita H0", "Não rejeita H0")

  data.frame(
    valor_p       = round(out$p_Wald, 4),
    decisao       = decisao,
    beta1_estimado= round(out$beta1_estimado, 4),
    beta1_real    = beta1_real,
    row.names     = rotulo
  )
}

res_A = exemplo_cenario(0.00, "Cenário A (H0 verdadeira, beta1=0.00)")
res_B = exemplo_cenario(0.15, "Cenário B (H0 falsa – efeito fraco, beta1=0.15)")
res_C = exemplo_cenario(0.60, "Cenário C (H0 falsa – efeito forte, beta1=0.60)")

tabela_exemplo = rbind(res_A, res_B, res_C)
noquote(tabela_exemplo)  # impressão 'limpa'

# ---- MONTE CARLO — tamanho e poder (Wald e LR) -----------------------
sim_glm_poisson = function(n, beta0, beta1, alpha, M) {
  x = runif(n, 0, 2)
  rej_W = rej_LR = numeric(M)

  for (m in 1:M) {
    lambda = exp(beta0 + beta1 * x)
    y      = rpois(n, lambda)
    out    = teste_glm_poisson(y, x)
    rej_W[m]  = as.integer(out$p_Wald < alpha)
    rej_LR[m] = as.integer(out$p_LR   < alpha)
  }
  data.frame(
    beta1 = beta1,
    tamanho_ou_poder_Wald = mean(rej_W),
    tamanho_ou_poder_LR   = mean(rej_LR)
  )
}

M = 2000
res_mc = rbind(
  sim_glm_poisson(n = 200, beta0 = 1, beta1 = 0.00, alpha = 0.05, M = M),  # tamanho nominal
  sim_glm_poisson(n = 200, beta0 = 1, beta1 = 0.15, alpha = 0.05, M = M),  # poder (fraco)
  sim_glm_poisson(n = 200, beta0 = 1, beta1 = 0.60, alpha = 0.05, M = M)   # poder (forte)
)
print(round(res_mc, 3))

     a H0_ideal H0_emp dep_emp hetero_emp
1 0.01     0.01  0.013   0.608      0.007
2 0.05     0.05  0.052   0.693      0.037
3 0.10     0.10  0.100   0.742      0.080
4 0.50     0.50  0.497   0.891      0.465


,valor_p,decisao,beta1_estimado,beta1_real
,<dbl>,<chr>,<dbl>,<dbl>
"Cenário A (H0 verdadeira, beta1=0.00)",0.2104,Não rejeita H0,-0.0992,0.00
"Cenário B (H0 falsa – efeito fraco, beta1=0.15)",0.1675,Não rejeita H0,0.0998,0.15
"Cenário C (H0 falsa – efeito forte, beta1=0.60)",0.0000,Rejeita H0,0.6242,0.60


  beta1 tamanho_ou_poder_Wald tamanho_ou_poder_LR
1  0.00                 0.044               0.045
2  0.15                 0.619               0.621
3  0.60                 1.000               1.000


In [3]:
# =====================================================================
# VALOR-P: (A) Poisson simples  |  (B) Regressão Poisson (GLM)
# =====================================================================

set.seed(123)

# =====================================================================
# A) POISSON SIMPLES — valor-p condicional a H0
# =====================================================================

sim_p_poisson = function(n, theta0, M, violacao = "nenhuma") {
  p = numeric(M)
  for (m in 1:M) {
    if (violacao == "nenhuma") {
      z = rpois(n, lambda = theta0)
    }
    if (violacao == "dependencia") {
      u = rnorm(1, 0, 0.25)
      z = rpois(n, lambda = pmax(1e-8, theta0 * exp(u)))
    }
    if (violacao == "hetero") {
      z = c(rpois(n/2, lambda = theta0 * 0.7),
            rpois(n - n/2, lambda = theta0 * 1.3))
    }
    Tn = n * ((mean(z) - theta0)^2 / var(z))
    p[m] = 1 - pchisq(Tn, df = 1)
  }
  return(p)
}

# parâmetros
n = 200
theta_real = 2
alpha = 0.05
M = 10000

# gera valores-p sob H0 e com desvios progressivos
H0_A = 2.00   # hipótese exata
H0_B = 2.06   # pequeno desvio
H0_C = 2.30   # grande desvio

teste_poisson = function(theta0, theta_real, n, alpha) {
  z = rpois(n, lambda = theta_real)
  Tn = n * ((mean(z) - theta0)^2 / var(z))
  p = 1 - pchisq(Tn, df = 1)
  decisao = ifelse(p < alpha, "Rejeita H0", "Não rejeita H0")
  data.frame(
    hipotese_nula = theta0,
    valor_p = round(p, 4),
    decisao = decisao,
    theta_estimado = round(mean(z), 3),
    theta_real = theta_real
  )
}

# executa três cenários
res_A = teste_poisson(H0_A, theta_real, n, alpha)
res_B = teste_poisson(H0_B, theta_real, n, alpha)
res_C = teste_poisson(H0_C, theta_real, n, alpha)

resumo_poisson = rbind(
  "Cenário A (H0 exata)"          = res_A,
  "Cenário B (H0 com pouco desvio)" = res_B,
  "Cenário C (H0 com muito desvio)" = res_C
)

cat("\n===== POISSON SIMPLES =====\n")
noquote(resumo_poisson)

# =====================================================================
# B) REGRESSÃO POISSON (GLM)
# =====================================================================

teste_glm_poisson = function(y, x) {
  m_full = glm(y ~ x, family = poisson)
  m_red  = glm(y ~ 1, family = poisson)
  b1 = coef(m_full)[2]
  se1 = sqrt(vcov(m_full)[2, 2])
  W  = (b1 / se1)^2
  pW = 1 - pchisq(W, df = 1)
  LR = 2 * (as.numeric(logLik(m_full)) - as.numeric(logLik(m_red)))
  pLR = 1 - pchisq(LR, df = 1)
  data.frame(pW = pW, pLR = pLR, b1 = b1, se1 = se1, W = W, LR = LR)
}

n = 200
beta0 = 1
alpha = 0.05
x = runif(n, 0, 2)

# ---- função para construir cenário com hipótese explícita ----
teste_cenario = function(beta1_real, alpha, rotulo) {
  lambda = exp(beta0 + beta1_real * x)
  y = rpois(n, lambda)
  out = teste_glm_poisson(y, x)
  decisao = ifelse(out$pW < alpha, "Rejeita H0", "Não rejeita H0")
  
  data.frame(
    hipotese_nula = "H0: beta1 = 0",
    valor_p = round(out$pW, 4),
    decisao = decisao,
    beta1_estimado = round(out$b1, 4),
    beta1_real = beta1_real,
    row.names = rotulo
  )
}

res_A = teste_cenario(0.00, alpha, "Cenário A (H0 verdadeira, beta1=0.00)")
res_B = teste_cenario(0.15, alpha, "Cenário B (H0 falsa – efeito fraco, beta1=0.15)")
res_C = teste_cenario(0.60, alpha, "Cenário C (H0 falsa – efeito forte, beta1=0.60)")

resumo_glm = rbind(res_A, res_B, res_C)

cat("\n===== REGRESSÃO POISSON (GLM) =====\n")
noquote(resumo_glm)


===== POISSON SIMPLES =====


,hipotese_nula,valor_p,decisao,theta_estimado,theta_real
,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Cenário A (H0 exata),2.00,0.6756,Não rejeita H0,2.04,2
Cenário B (H0 com pouco desvio),2.06,0.4446,Não rejeita H0,1.98,2
Cenário C (H0 com muito desvio),2.30,0.0065,Rejeita H0,2.02,2



===== REGRESSÃO POISSON (GLM) =====


,hipotese_nula,valor_p,decisao,beta1_estimado,beta1_real
,<chr>,<dbl>,<chr>,<dbl>,<dbl>
"Cenário A (H0 verdadeira, beta1=0.00)",H0: beta1 = 0,0.4891,Não rejeita H0,0.0514,0.00
"Cenário B (H0 falsa – efeito fraco, beta1=0.15)",H0: beta1 = 0,0.0891,Não rejeita H0,0.1184,0.15
"Cenário C (H0 falsa – efeito forte, beta1=0.60)",H0: beta1 = 0,0.0000,Rejeita H0,0.6125,0.60


In [13]:
# ====================================================================
# Regressão Poisson (GLM)
# ====================================================================

set.seed(111)

# Parâmetros
n = 200
beta0 = 1.0
beta1_real = 0.20   # efeito verdadeiro fixo
alpha = 0.05

# Covariável e única amostra gerada do mundo real
x = runif(n, 0, 2)
lambda = exp(beta0 + beta1_real * x)
y = rpois(n, lambda)

# Ajuste único nos mesmos dados
modelo = glm(y ~ x, family = poisson)

# ---------------------------------------------------------------
# Função: testa H0: beta1 = b0 (teste de Wald)
# ---------------------------------------------------------------
testa_glm_1amostra = function(mod, b0) {
  b1_hat = coef(mod)[2]
  se_b1  = sqrt(vcov(mod)[2, 2])
  Z = (b1_hat - b0) / se_b1
  p_valor = 2 * (1 - pnorm(abs(Z)))   # bicaudal
  decisao = ifelse(p_valor < alpha, "Rejeita H0", "Não rejeita H0")
  data.frame(
    hipotese_nula = paste0("H0: beta1 = ", format(b0, nsmall = 2)),
    valor_p = p_valor,
    decisao = decisao,
    beta1_estimado = b1_hat,
    beta1_real = beta1_real
  )
}

# Hipóteses a testar sobre o mesmo conjunto (exato, pouco e muito desvio)
B0s = c(0.20, 0.10, 0.60)

resA = testa_glm_1amostra(modelo, B0s[1])  # exato 
resB = testa_glm_1amostra(modelo, B0s[2])  # pouco desvio
resC = testa_glm_1amostra(modelo, B0s[3])  # desvio forte

resumo_glm = rbind(
  "Cenário A (H0 exato)"              = resA,
  "Cenário B (H0 com pouco desvio)"   = resB,
  "Cenário C (H0 com muito desvio)"   = resC
)

# Desativa notação científica
options(scipen = 999)

# Arredonda apenas as colunas numéricas
resumo_glm_round <- resumo_glm
resumo_glm_round$valor_p <- round(resumo_glm_round$valor_p, 6)
resumo_glm_round$beta1_estimado <- round(resumo_glm_round$beta1_estimado, 6)
resumo_glm_round$beta1_real <- round(resumo_glm_round$beta1_real, 6)

# Formata as saídas
resumo_glm_format <- data.frame(
  hipotese_nula = resumo_glm_round$hipotese_nula,
  valor_p = format(resumo_glm_round$valor_p, nsmall = 6),
  decisao = resumo_glm_round$decisao,
  beta1_estimado = format(resumo_glm_round$beta1_estimado, nsmall = 6),
  beta1_real = format(resumo_glm_round$beta1_real, nsmall = 1)
)

# Tabela com os resultados
noquote(resumo_glm_format)

hipotese_nula,valor_p,decisao,beta1_estimado,beta1_real
<chr>,<chr>,<chr>,<chr>,<chr>
H0: beta1 = 0.20,0.425228,Não rejeita H0,0.254359,0.2
H0: beta1 = 0.10,0.023557,Rejeita H0,0.254359,0.2
H0: beta1 = 0.60,0.000000,Rejeita H0,0.254359,0.2


In [16]:
# ====================================================================
# Regressão Poisson (GLM) 
# ====================================================================

set.seed(111)

# Parâmetros fixos do "mundo real"
n      = 200
beta0  = 1.0
theta  = 0.20     # valor verdadeiro do parâmetro (inclinação)
alpha  = 0.05

# Covariável e única amostra observada
x       = runif(n, 0, 2)
lambda  = exp(beta0 + theta * x)
y       = rpois(n, lambda)

# Ajuste único do GLM Poisson nos MESMOS dados
modelo  = glm(y ~ x, family = poisson)

# ---------------------------------------------------------------
# Função: testa H0: theta = h0 (Wald)
# ---------------------------------------------------------------
testa_glm_theta = function(mod, h0, alpha = 0.05) {
  theta_hat = coef(mod)[2]
  se_hat    = sqrt(vcov(mod)[2, 2])
  Z         = (theta_hat - h0) / se_hat

  # valor-p:
  p_valor   = 1 - pnorm(Z)

  decisao   = ifelse(p_valor < alpha, "Rejeita H0", "Não rejeita H0")

  data.frame(
    hipotese_nula = paste0("H0: theta = ", format(h0, nsmall = 2)),
    valor_p       = p_valor,
    decisao       = decisao,
    theta_hat     = theta_hat,
    theta         = theta
  )
}

# Hipóteses nulas a testar no MESMO conjunto (exato, pouco e muito desvio)
H0_A = 0.20   # H0 exato
H0_B = 0.10   # H0 com pouco desvio
H0_C = 0.60   # H0 com muito desvio

resA = testa_glm_theta(modelo, H0_A, alpha)
resB = testa_glm_theta(modelo, H0_B, alpha)
resC = testa_glm_theta(modelo, H0_C, alpha)

# Consolida com rbind (mantendo os rótulos de cenário como nomes de linha)
resumo_glm = rbind(
  "Cenário A (H0 exato)"            = resA,
  "Cenário B (H0 pouco desvio)"     = resB,
  "Cenário C (H0 muito desvio)"     = resC
)

# -------- Formatação (sem notação científica e 6 casas dicimais) --------
options(scipen = 999)

resumo_fmt          = resumo_glm
resumo_fmt$valor_p  = format(round(resumo_fmt$valor_p,  6), nsmall = 6)
resumo_fmt$theta_hat= format(round(resumo_fmt$theta_hat,6), nsmall = 6)
resumo_fmt$theta    = format(round(resumo_fmt$theta,    6), nsmall = 6)

# Tabela final
noquote(resumo_fmt)

,hipotese_nula,valor_p,decisao,theta_hat,theta
,<chr>,<chr>,<chr>,<chr>,<chr>
Cenário A (H0 exato),H0: theta = 0.20,0.212614,Não rejeita H0,0.254359,0.200000
Cenário B (H0 pouco desvio),H0: theta = 0.10,0.011779,Rejeita H0,0.254359,0.200000
Cenário C (H0 muito desvio),H0: theta = 0.60,1.000000,Não rejeita H0,0.254359,0.200000
